# MLB Data Cleaning
Source: https://sabr.org/lahman-database/
In this notebook we will be inspecting the data structures of multiple files, cleaning and processing for use in SQL queries.
- Section 1: People.csv – Handling missing values and unnecessary columns.  
- Section 2: Teams.csv – Cleaning team statistics and removing irrelevant fields.  
- Section 3: Batting.csv – Fixing missing values, dropping bad columns.  
- Section 4: Pitching.csv – Cleaning incomplete records, ensuring data consistency.  
- Section 5: Salaries.csv – Checking for duplicate records.  
- Final Step: Save Cleaned Data – Export cleaned versions as CSV files for SQLite.  

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **Batting**

In [3]:
import pandas as pd

# Load Batting.csv
batting = pd.read_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/lahman_1871-2023_csv/Batting.csv", encoding="latin1")
batting = batting[batting["yearID"] >= 2000]

# 📌 Display first few rows
print("📊 First 5 Rows of Batting.csv:")
display(batting.head())

📊 First 5 Rows of Batting.csv:


,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
0,aardsda01,2004,1,SFN,NL,11,NaN,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,aardsda01,2006,1,CHN,NL,45,NaN,2,0,0,...,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
2,aardsda01,2007,1,CHA,AL,25,NaN,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,aardsda01,2008,1,BOS,AL,47,NaN,1,0,0,...,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
4,aardsda01,2009,1,SEA,AL,73,NaN,0,0,0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [4]:
# 📌 Display column names
print("\n🛠 Column Names:")
display(batting.columns)

# 📌 Compare columns to the README file (Expected columns)
expected_columns = [
    "playerID", "yearID", "stint", "teamID", "lgID", "G", "AB", "R", "H", "2B",
    "3B", "HR", "RBI", "SB", "CS", "BB", "SO", "IBB", "HBP", "SH", "SF", "GIDP"
]

# Identify unexpected columns
unexpected_columns = [col for col in batting.columns if col not in expected_columns]
if unexpected_columns:
    print("\n🚨 Unexpected Columns Found (Not in README):")
    display(unexpected_columns)
else:
    print("\n✅ All columns match the README file!")


🛠 Column Names:


Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'G_batting', 'AB',
       'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP',
       'SH', 'SF', 'GIDP', 'G_old'],
      dtype='object')


🚨 Unexpected Columns Found (Not in README):


['G_batting', 'G_old']

In [5]:
# 📌 Check missing values
missing_batting = batting.isnull().sum()
print("\n⚠️ Missing Values Per Column:")
display(missing_batting[missing_batting > 0])

# 📌 Check for duplicate player stats for the same team and year
duplicate_stats = batting[batting.duplicated(subset=["playerID", "yearID", "teamID", "stint"], keep=False)]
print("\n🔍 Players with Duplicate Stats for the Same Team in the Same Year:")
display(duplicate_stats)

# 📌 Ensure `stint` makes sense for players who played for multiple teams in a season
stint_issues = batting.groupby(["playerID", "yearID"]).agg({"stint": "nunique"})
stint_issues = stint_issues[stint_issues["stint"] > 1]
print("\n🔍 Players Who Played for Multiple Teams in a Year (Check if `stint` is valid):")
display(stint_issues)

# 📌 Check for negative values (should not exist in baseball stats)
negative_values = batting.select_dtypes(include=["number"]).lt(0).sum()
print("\n🚨 Columns with Negative Values:")
display(negative_values[negative_values > 0])

# 📌 Check if all players have at least 1 at-bat (AB > 0) if they have any hits (H > 0)
invalid_at_bats = batting[(batting["H"] > 0) & (batting["AB"] == 0)]
print("\n❌ Players with Hits but 0 At-Bats (Shouldn't Happen):")
display(invalid_at_bats)


⚠️ Missing Values Per Column:


G_batting    32918
G_old        34533
dtype: int64


🔍 Players with Duplicate Stats for the Same Team in the Same Year:


,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old



🔍 Players Who Played for Multiple Teams in a Year (Check if `stint` is valid):


stint
playerID  yearID       
abadfe01  2016        2
abbotpa01 2004        2
abernbr01 2003        2
abramcj01 2022        2
abreual01 2022        3
...                 ...
zieglbr01 2011        2
          2016        2
          2018        2
zimmebr01 2022        3
zobribe01 2015        2

[2844 rows x 1 columns]


🚨 Columns with Negative Values:


Series([], dtype: int64)


❌ Players with Hits but 0 At-Bats (Shouldn't Happen):


,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old


In [6]:
# 📌 Summary statistics to check for outliers
print("\n📈 Summary Statistics:")
display(batting.describe())


📈 Summary Statistics:


,yearID,stint,G,G_batting,AB,R,H,2B,3B,HR,...,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,G_old
count,34533.000000,34533.000000,34533.000000,1615.000000,34533.000000,34533.000000,34533.000000,34533.000000,34533.000000,34533.000000,...,34533.000000,34533.000000,34533.000000,34533.000000,34533.000000,34533.000000,34533.000000,34533.000000,34533.000000,0.0
mean,2011.927287,1.093273,47.229867,31.424149,112.146758,15.000261,28.876032,5.767034,0.581009,3.573683,...,1.861755,0.713260,10.592303,24.297484,0.726117,1.197029,0.870848,0.874844,2.500391,NaN
std,6.979684,0.311214,44.758711,48.662969,174.827916,25.885189,48.170159,10.023124,1.455214,7.418943,...,5.363428,1.795191,19.239762,37.120066,2.355530,2.644883,2.033984,1.778274,4.513001,NaN
min,2000.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,2006.000000,1.000000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,2012.000000,1.000000,31.000000,1.000000,12.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,2018.000000,1.000000,69.000000,49.500000,160.000000,19.000000,38.000000,7.000000,0.000000,3.000000,...,1.000000,0.000000,13.000000,35.000000,0.000000,1.000000,1.000000,1.000000,3.000000,NaN
max,2023.000000,5.000000,163.000000,162.000000,716.000000,152.000000,262.000000,59.000000,23.000000,73.000000,...,78.000000,24.000000,232.000000,223.000000,120.000000,34.000000,24.000000,16.000000,32.000000,NaN


In [7]:
# 📌 1️⃣ Drop Unnecessary Columns
batting.drop(columns=["G_batting", "G_old"], inplace=True, errors="ignore")

# 📌 2️⃣ Save Cleaned Data
batting.to_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/cleaned_and_processed_data/Batting_Cleaned_Final.csv", index=False)

print("✅ `Batting.csv` cleaned and saved!")

✅ `Batting.csv` cleaned and saved!


# **Pitching**

In [8]:
# Load Pitching.csv
pitching = pd.read_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/lahman_1871-2023_csv/Pitching.csv", encoding="latin1")
pitching = pitching[pitching["yearID"] >= 2000]

# 📌 Display first few rows
print("📊 First 5 Rows of Pitching.csv:")
display(pitching.head())

📊 First 5 Rows of Pitching.csv:


,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,aardsda01,2004,1,SFN,NL,1,0,11,0,0,...,0.0,0,2.0,0,61.0,5,8,0.0,1.0,1.0
1,aardsda01,2006,1,CHN,NL,3,0,45,0,0,...,0.0,1,1.0,0,225.0,9,25,1.0,3.0,2.0
2,aardsda01,2007,1,CHA,AL,2,1,25,0,0,...,3.0,2,1.0,0,151.0,7,24,2.0,1.0,1.0
3,aardsda01,2008,1,BOS,AL,4,2,47,0,0,...,2.0,3,5.0,0,228.0,7,32,3.0,2.0,4.0
4,aardsda01,2009,1,SEA,AL,3,6,73,0,0,...,3.0,2,0.0,0,296.0,53,23,2.0,1.0,2.0


In [9]:
# 📌 Display column names
print("\n🛠 Column Names in Pitching.csv:")
display(pitching.columns)

# 📌 Compare columns to the README file (Expected columns)
expected_columns = [
    "playerID", "yearID", "stint", "teamID", "lgID", "W", "L", "G", "GS", "CG", "SHO", "SV", "IPouts",
    "H", "ER", "HR", "BB", "SO", "BAOpp", "ERA", "IBB", "WP", "HBP", "BK", "BFP", "GF", "R", "SH", "SF", "GIDP"
]

# Identify unexpected columns
unexpected_columns = [col for col in pitching.columns if col not in expected_columns]
if unexpected_columns:
    print("\n🚨 Unexpected Columns Found (Not in README):")
    display(unexpected_columns)
else:
    print("\n✅ All columns match the README file!")


🛠 Column Names in Pitching.csv:


Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'W', 'L', 'G', 'GS',
       'CG', 'SHO', 'SV', 'IPouts', 'H', 'ER', 'HR', 'BB', 'SO', 'BAOpp',
       'ERA', 'IBB', 'WP', 'HBP', 'BK', 'BFP', 'GF', 'R', 'SH', 'SF', 'GIDP'],
      dtype='object')


✅ All columns match the README file!


In [10]:
# 📌 Identify missing values
missing_values = pitching.isnull().sum()
missing_cols = missing_values[missing_values > 0].index.tolist()

# 📌 Display only columns with missing values
print("\n⚠️ Columns with Missing Data:")
display(missing_values[missing_values > 0])

# 📌 Inspect rows where data is missing for each column
for col in missing_cols:
    print(f"\n🔍 Inspecting Rows Where `{col}` is Missing:")
    display(pitching[pitching[col].isnull()].head(10))  # Display first 10 affected rows


⚠️ Columns with Missing Data:


BAOpp    11
ERA      22
GIDP      3
dtype: int64


🔍 Inspecting Rows Where `BAOpp` is Missing:


,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
5283,brothre01,2018,1,ATL,NL,0,0,1,0,0,...,0.0,0,0.0,0,2.0,0,1,0.0,0.0,0.0
12298,dunnija01,2014,1,SFN,NL,0,0,1,0,0,...,0.0,2,0.0,0,2.0,0,0,0.0,1.0,0.0
13132,eschja01,2017,1,SDN,NL,0,0,1,0,0,...,0.0,0,0.0,0,2.0,0,0,0.0,0.0,0.0
18328,haltesh01,2000,1,DET,AL,0,0,1,0,0,...,0.0,0,0.0,0,1.0,0,0,0.0,0.0,0.0
19584,hembrhe01,2021,1,NYN,NL,0,0,15,0,0,...,1.0,1,0.0,0,64.0,5,6,0.0,0.0,NaN
19585,hembrhe01,2021,2,CIN,NL,2,7,45,0,0,...,0.0,1,2.0,0,179.0,16,33,0.0,0.0,NaN
37434,radinsc01,2000,1,SLN,NL,0,0,1,0,0,...,0.0,0,0.0,0,1.0,0,0,0.0,0.0,0.0
39617,romansa01,2021,3,NYA,AL,0,1,4,0,0,...,0.0,0,1.0,0,20.0,0,2,0.0,0.0,NaN
46302,tollesh01,2013,1,LAN,NL,0,0,1,0,0,...,0.0,0,0.0,0,2.0,0,0,0.0,0.0,0.0
47312,vasquan02,2019,1,MIN,AL,0,0,1,0,0,...,0.0,0,1.0,0,3.0,0,3,0.0,0.0,0.0



🔍 Inspecting Rows Where `ERA` is Missing:


,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
5283,brothre01,2018,1,ATL,NL,0,0,1,0,0,...,0.0,0,0.0,0,2.0,0,1,0.0,0.0,0.0
7976,choatra01,2007,1,ARI,NL,0,0,2,0,0,...,0.0,0,0.0,0,3.0,0,0,0.0,0.0,0.0
13132,eschja01,2017,1,SDN,NL,0,0,1,0,0,...,0.0,0,0.0,0,2.0,0,0,0.0,0.0,0.0
14360,florebe01,2021,1,SLN,NL,0,0,1,0,0,...,0.0,0,0.0,0,3.0,0,1,0.0,0.0,0.0
17201,greenni02,2015,1,SLN,NL,0,1,1,0,0,...,0.0,0,0.0,0,2.0,1,2,0.0,0.0,0.0
18328,haltesh01,2000,1,DET,AL,0,0,1,0,0,...,0.0,0,0.0,0,1.0,0,0,0.0,0.0,0.0
20463,hillri01,2014,1,LAA,AL,0,0,2,0,0,...,0.0,1,0.0,0,4.0,0,1,0.0,0.0,0.0
22990,jonesna01,2014,1,CHA,AL,0,0,2,0,0,...,0.0,0,0.0,0,5.0,0,4,0.0,0.0,0.0
24624,kochma01,2017,1,ARI,NL,0,0,1,0,0,...,0.0,0,0.0,0,3.0,0,3,0.0,0.0,0.0
29476,mccutda01,2012,1,PIT,NL,0,1,1,0,0,...,0.0,0,0.0,0,2.0,1,2,0.0,0.0,0.0



🔍 Inspecting Rows Where `GIDP` is Missing:


,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
19584,hembrhe01,2021,1,NYN,NL,0,0,15,0,0,...,1.0,1,0.0,0,64.0,5,6,0.0,0.0,NaN
19585,hembrhe01,2021,2,CIN,NL,2,7,45,0,0,...,0.0,1,2.0,0,179.0,16,33,0.0,0.0,NaN
39617,romansa01,2021,3,NYA,AL,0,1,4,0,0,...,0.0,0,1.0,0,20.0,0,2,0.0,0.0,NaN


In [11]:
# 📌 Summary statistics for salary data
print("\n📈 Salary Summary Statistics:")
display(pitching.describe())


📈 Salary Summary Statistics:


,yearID,stint,W,L,G,GS,CG,SHO,SV,IPouts,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
count,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,...,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18464.000000,18461.000000
mean,2012.376300,1.094887,3.074578,3.074578,24.500921,6.149805,0.158687,0.066183,1.547552,164.372888,...,1.358048,2.065045,2.238789,0.194487,235.103661,5.991118,28.054809,1.628737,1.636211,4.676941
std,6.997603,0.312390,4.010145,3.529683,20.584397,10.254455,0.637482,0.316470,6.022005,170.437879,...,1.833300,2.539797,2.729239,0.513934,239.072472,10.189901,28.184810,2.419119,1.974481,5.547916
min,2000.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2006.000000,1.000000,0.000000,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,37.000000,...,0.000000,0.000000,0.000000,0.000000,56.000000,0.000000,8.000000,0.000000,0.000000,1.000000
50%,2013.000000,1.000000,2.000000,2.000000,20.000000,0.000000,0.000000,0.000000,0.000000,108.000000,...,1.000000,1.000000,1.000000,0.000000,157.000000,2.000000,19.000000,1.000000,1.000000,3.000000
75%,2019.000000,1.000000,4.000000,5.000000,34.000000,8.000000,0.000000,0.000000,0.000000,214.000000,...,2.000000,3.000000,3.000000,0.000000,303.000000,8.000000,36.000000,2.000000,2.000000,6.000000
max,2023.000000,5.000000,24.000000,21.000000,94.000000,36.000000,11.000000,6.000000,62.000000,798.000000,...,17.000000,25.000000,24.000000,6.000000,1071.000000,79.000000,152.000000,21.000000,14.000000,40.000000


In [12]:
# 📌 Fill Missing ERA (If We Have ER and IPouts)
era_mask = pitching["ERA"].isnull() & pitching["ER"].notnull() & pitching["IPouts"].notnull()
pitching.loc[era_mask, "ERA"] = (pitching["ER"] / (pitching["IPouts"] / 3)) * 9
print(f"✅ Filled {era_mask.sum()} missing ERA values.")

# 📌 Handle Infinite ERA Cases (ER > 0 and IPouts == 0)
inf_era_mask = (pitching["ERA"].isnull()) & (pitching["ER"] > 0) & (pitching["IPouts"] == 0)
pitching.loc[inf_era_mask, "ERA"] = "INF"

# Handle where ER is 0
no_era_mask = (pitching["ERA"].isnull())
pitching.loc[no_era_mask, "ERA"] = 0

# 📌 Fill Missing BAA (If We Have Hits and BFP)
baa_mask = pitching["BAOpp"].isnull() & pitching["H"].notnull() & pitching["BFP"].notnull()
pitching.loc[baa_mask, "BAOpp"] = pitching["H"] / (pitching["BFP"] - pitching["BB"] - pitching["HBP"] - pitching["IBB"])
print(f"✅ Filled {baa_mask.sum()} missing Opponent BAA values.")

# 📌 Handle Cases Where BAA Should Be 0 (No Hits, No At-Bats)
zero_baa_mask = (pitching["BAOpp"].isnull()) & (pitching["H"] == 0)
pitching.loc[zero_baa_mask, "BAOpp"] = 0

pitching["BAOpp"] = pitching["BAOpp"].astype(float)

# 📌 Fill Missing `GIDP` with `0`
pitching["GIDP"].fillna(0, inplace=True)
print(f"✅ Replaced {pitching['GIDP'].isnull().sum()} missing GIDP values with 0.")

✅ Filled 22 missing ERA values.
✅ Filled 11 missing Opponent BAA values.
✅ Replaced 0 missing GIDP values with 0.


In [13]:
# 📌 Check if there are still missing values in ERA, BAA, or GIDP
missing_values = pitching[["ERA", "BAOpp", "GIDP"]].isnull().sum()
print("\n⚠️ Missing Values After Cleaning:")
display(missing_values)


⚠️ Missing Values After Cleaning:


ERA      0
BAOpp    0
GIDP     0
dtype: int64

In [14]:
# 📌 Check the statistical summary for ERA, BAA, and GIDP
print("\n📊 Summary Statistics After Cleaning:")
display(pitching[["ERA", "BAOpp", "GIDP"]].describe())


📊 Summary Statistics After Cleaning:


,BAOpp,GIDP
count,18464.000000,18464.000000
mean,0.269216,4.676181
std,0.085758,5.547786
min,0.000000,0.000000
25%,0.228000,1.000000
50%,0.261000,3.000000
75%,0.298000,6.000000
max,1.000000,40.000000


In [15]:
pitching.to_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/cleaned_and_processed_data/Pitching_Cleaned_Final.csv", index=False)

print("✅ `Pitching.csv` cleaned and saved!")

✅ `Pitching.csv` cleaned and saved!


# **People**

In [16]:
# Load People.csv
people = pd.read_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/lahman_1871-2023_csv/People.csv", encoding="latin1")
people = people[people["finalGame"] >= "2000-01-01"]

# 📌 Display first few rows
print("📊 First 5 Rows of People.csv:")
display(people.head())

📊 First 5 Rows of People.csv:


,ID,playerID,birthYear,birthMonth,birthDay,birthCity,birthCountry,birthState,deathYear,deathMonth,...,nameLast,nameGiven,weight,height,bats,throws,debut,bbrefID,finalGame,retroID
0,1,aardsda01,1981.0,12.0,27.0,Denver,USA,CO,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,aardsda01,2015-08-23,aardd001
4,5,abadan01,1972.0,8.0,25.0,Palm Beach,USA,FL,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,abadan01,2006-04-13,abada001
5,6,abadfe01,1985.0,12.0,17.0,La Romana,D.R.,La Romana,NaN,NaN,...,Abad,Fernando Antonio,235.0,74.0,L,L,2010-07-28,abadfe01,2021-10-01,abadf001
10,11,abbotco01,1995.0,9.0,20.0,San Diego,USA,CA,NaN,NaN,...,Abbott,Cory James,220.0,74.0,R,R,2021-06-05,abbotco01,2022-10-04,abboc001
14,15,abbotje01,1972.0,8.0,17.0,Atlanta,USA,GA,NaN,NaN,...,Abbott,Jeffrey William,190.0,74.0,R,L,1997-06-10,abbotje01,2001-09-29,abboj002


In [17]:
# 📌 Display column names
print("\n🛠 Column Names in People.csv:")
display(people.columns)

# 📌 Compare columns to the README file (Expected columns)
expected_columns = [
    "playerID", "birthYear", "birthMonth", "birthDay", "birthCountry", "birthState", "birthCity",
    "deathYear", "deathMonth", "deathDay", "deathCountry", "deathState", "deathCity",
    "nameFirst", "nameLast", "nameGiven", "weight", "height", "bats", "throws",
    "debut", "finalGame", "retroID", "bbrefID"
]

# Identify unexpected columns
unexpected_columns = [col for col in people.columns if col not in expected_columns]
if unexpected_columns:
    print("\n🚨 Unexpected Columns Found (Not in README):")
    display(unexpected_columns)
else:
    print("\n✅ All columns match the README file!")


🛠 Column Names in People.csv:


Index(['ID', 'playerID', 'birthYear', 'birthMonth', 'birthDay', 'birthCity',
       'birthCountry', 'birthState', 'deathYear', 'deathMonth', 'deathDay',
       'deathCountry', 'deathState', 'deathCity', 'nameFirst', 'nameLast',
       'nameGiven', 'weight', 'height', 'bats', 'throws', 'debut', 'bbrefID',
       'finalGame', 'retroID'],
      dtype='object')


🚨 Unexpected Columns Found (Not in README):


['ID']

In [18]:
# 📌 Check missing values
missing_people = people.isnull().sum()
print("\n⚠️ Missing Values Per Column:")
display(missing_people[missing_people > 0])

# 📌 Check for duplicate `playerID`
duplicate_players = people[people.duplicated(subset=["playerID"], keep=False)]
print("\n🔍 Duplicate PlayerIDs (Should be 0):")
display(duplicate_players)

# 📌 Check for unrealistic values (e.g., extreme height, weight)
print("\n🚨 Unusual Height and Weight Values:")
display(people[(people["height"] > 90) | (people["height"] < 50)])
display(people[(people["weight"] > 400) | (people["weight"] < 100)])


⚠️ Missing Values Per Column:


birthCity          1
birthState       210
deathYear       6595
deathMonth      6595
deathDay        6595
deathCountry    6595
deathState      6598
deathCity       6595
dtype: int64


🔍 Duplicate PlayerIDs (Should be 0):


,ID,playerID,birthYear,birthMonth,birthDay,birthCity,birthCountry,birthState,deathYear,deathMonth,...,nameLast,nameGiven,weight,height,bats,throws,debut,bbrefID,finalGame,retroID



🚨 Unusual Height and Weight Values:


,ID,playerID,birthYear,birthMonth,birthDay,birthCity,birthCountry,birthState,deathYear,deathMonth,...,nameLast,nameGiven,weight,height,bats,throws,debut,bbrefID,finalGame,retroID


,ID,playerID,birthYear,birthMonth,birthDay,birthCity,birthCountry,birthState,deathYear,deathMonth,...,nameLast,nameGiven,weight,height,bats,throws,debut,bbrefID,finalGame,retroID


In [19]:
# 📌 Summary statistics for numerical values
print("\n📈 Summary Statistics:")
display(people.describe())


📈 Summary Statistics:


,ID,birthYear,birthMonth,birthDay,deathYear,deathMonth,deathDay,weight,height
count,6667.000000,6667.000000,6667.000000,6667.000000,72.000000,72.000000,72.000000,6667.000000,6667.000000
mean,10781.100495,1984.282736,6.649918,15.767212,2015.097222,6.986111,14.722222,205.840258,73.641668
std,6235.573121,9.241371,3.423786,8.748449,6.244231,3.644378,8.993303,22.062736,2.308847
min,1.000000,1957.000000,1.000000,1.000000,2001.000000,1.000000,1.000000,140.000000,66.000000
25%,5392.500000,1977.000000,4.000000,8.000000,2010.750000,3.750000,7.000000,190.000000,72.000000
50%,10989.000000,1985.000000,7.000000,16.000000,2017.000000,7.000000,14.500000,205.000000,74.000000
75%,16076.500000,1992.000000,10.000000,23.000000,2020.250000,10.000000,22.250000,220.000000,75.000000
max,20937.000000,2003.000000,12.000000,31.000000,2023.000000,12.000000,31.000000,320.000000,83.000000


In [20]:
# 📌 1️⃣ Drop Unnecessary Columns (All Death-Related)
people.drop(columns=["ID", "deathYear", "deathMonth", "deathDay", 
                     "deathCountry", "deathState", "deathCity"], inplace=True, errors="ignore")

# 📌 2️⃣ Convert `finalGame` to datetime format
people["finalGame"] = pd.to_datetime(people["finalGame"], errors="coerce")

# 📌 4️⃣ Fill Missing `birthState` with `"Unknown"` for Non-USA Players
people.loc[people["birthCountry"] != "USA", "birthState"] = \
    people["birthState"].fillna("Unknown")

In [21]:
people.to_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/cleaned_and_processed_data/People_Cleaned_Final.csv", index=False)

print("✅ `People.csv` cleaned and saved!")

✅ `People.csv` cleaned and saved!


# **Teams**

In [22]:
teams = pd.read_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/lahman_1871-2023_csv/Teams.csv", encoding="latin1")

# Filter to 2000+
teams = teams[teams["yearID"] >= 2000]

# 📌 Display first few rows
print("📊 First 5 Rows of Teams.csv:")
display(teams.head())

📊 First 5 Rows of Teams.csv:


,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
2325,2000,AL,ANA,ANA,W,3,162,81.0,82,80,...,182,0.978,Anaheim Angels,Edison International Field,2066982.0,102,103,ANA,ANA,ANA
2326,2000,AL,BAL,BAL,E,4,162,81.0,74,88,...,151,0.981,Baltimore Orioles,Oriole Park at Camden Yards,3297031.0,95,96,BAL,BAL,BAL
2327,2000,AL,BOS,BOS,E,2,162,81.0,85,77,...,120,0.982,Boston Red Sox,Fenway Park II,2585895.0,104,103,BOS,BOS,BOS
2328,2000,AL,CHA,CHW,C,1,162,81.0,95,67,...,190,0.978,Chicago White Sox,Comiskey Park II,1947799.0,102,102,CHW,CHA,CHA
2329,2000,AL,CLE,CLE,C,2,162,81.0,90,72,...,147,0.988,Cleveland Indians,Jacobs Field,3456278.0,101,100,CLE,CLE,CLE


In [23]:
# 📌 Display column names
print("\n🛠 Column Names in Teams.csv:")
display(teams.columns)

# 📌 Compare columns to the README file (Expected columns)
expected_columns = [
    "yearID", "lgID", "teamID", "franchID", "divID", "Rank", "G", "Ghome", "W", "L", 
    "DivWin", "WCWin", "LgWin", "WSWin", "R", "AB", "H", "2B", "3B", "HR", "BB", "SO", 
    "SB", "CS", "HBP", "SF", "RA", "ER", "ERA", "CG", "SHO", "SV", "IPouts", "HA", "HRA", 
    "BBA", "SOA", "E", "DP", "FP", "name", "park", "attendance", "BPF", "PPF", 
    "teamIDBR", "teamIDlahman45", "teamIDretro"
]

# Identify unexpected columns
unexpected_columns = [col for col in teams.columns if col not in expected_columns]
if unexpected_columns:
    print("\n🚨 Unexpected Columns Found (Not in README):")
    display(unexpected_columns)
else:
    print("\n✅ All columns match the README file!")


🛠 Column Names in Teams.csv:


Index(['yearID', 'lgID', 'teamID', 'franchID', 'divID', 'Rank', 'G', 'Ghome',
       'W', 'L', 'DivWin', 'WCWin', 'LgWin', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'CS', 'HBP', 'SF', 'RA', 'ER', 'ERA',
       'CG', 'SHO', 'SV', 'IPouts', 'HA', 'HRA', 'BBA', 'SOA', 'E', 'DP', 'FP',
       'name', 'park', 'attendance', 'BPF', 'PPF', 'teamIDBR',
       'teamIDlahman45', 'teamIDretro'],
      dtype='object')


✅ All columns match the README file!


In [24]:
# 📌 Check missing values
missing_teams = teams.isnull().sum()
print("\n⚠️ Missing Values Per Column:")
display(missing_teams[missing_teams > 0])

# 📌 Check for duplicate team records per year
duplicate_teams = teams[teams.duplicated(subset=["yearID", "teamID"], keep=False)]
print("\n🔍 Duplicate Teams for the Same Year (Should be 0):")
display(duplicate_teams)

# 📌 Ensure `W` + `L` ≈ `G` (Wins + Losses should match Games Played)
teams["W_L_Check"] = teams["W"] + teams["L"]
teams["Games_Check"] = teams["G"]
mismatched_records = teams[teams["W_L_Check"] != teams["Games_Check"]]
print("\n🚨 Records Where Wins + Losses ≠ Games Played:")
display(mismatched_records)


⚠️ Missing Values Per Column:


Series([], dtype: int64)


🔍 Duplicate Teams for the Same Year (Should be 0):


,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro



🚨 Records Where Wins + Losses ≠ Games Played:


,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro,W_L_Check,Games_Check
2342,2000,NL,CIN,CIN,C,2,163,82.0,85,77,...,Cincinnati Reds,Cinergy Field,2577371.0,102,102,CIN,CIN,CIN,162,163
2347,2000,NL,MIL,MIL,C,3,163,81.0,73,89,...,Milwaukee Brewers,County Stadium,1573621.0,97,98,MIL,ML4,MIL,162,163
2356,2001,AL,BAL,BAL,E,4,162,80.0,63,98,...,Baltimore Orioles,Oriole Park at Camden Yards,3094841.0,95,96,BAL,BAL,BAL,161,162
2363,2001,AL,NYA,NYY,E,1,161,80.0,95,65,...,New York Yankees,Yankee Stadium II,3264907.0,102,100,NYY,NYA,NYA,160,161
2400,2002,NL,ATL,ATL,E,1,161,81.0,101,59,...,Atlanta Braves,Turner Field,2603484.0,102,101,ATL,ATL,ATL,160,161
2413,2002,NL,SFN,SFG,W,2,162,81.0,95,66,...,San Francisco Giants,PacBell Park,3253203.0,95,94,SFG,SFN,SFN,161,162
2416,2003,AL,BAL,BAL,E,4,163,81.0,71,91,...,Baltimore Orioles,Oriole Park at Camden Yards,2454523.0,99,99,BAL,BAL,BAL,162,163
2423,2003,AL,NYA,NYY,E,1,163,82.0,101,61,...,New York Yankees,Yankee Stadium II,3465600.0,98,97,NYY,NYA,NYA,162,163
2492,2005,NL,CIN,CIN,C,5,163,82.0,73,89,...,Cincinnati Reds,Great American Ball Park,1943067.0,100,101,CIN,CIN,CIN,162,163
2495,2005,NL,HOU,HOU,C,2,163,81.0,89,73,...,Houston Astros,Minute Maid Park,2804760.0,101,100,HOU,HOU,HOU,162,163


In [25]:
# 📌 Summary statistics for numerical values
print("\n📈 Summary Statistics:")
display(teams.describe())


📈 Summary Statistics:


,yearID,Rank,G,Ghome,W,L,R,AB,H,2B,...,BBA,SOA,E,DP,FP,attendance,BPF,PPF,W_L_Check,Games_Check
count,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,720.000000,...,720.000000,720.000000,720.000000,720.000000,720.000000,7.200000e+02,720.000000,720.000000,720.000000,720.000000
mean,2011.500000,3.006944,157.708333,78.837500,78.845833,78.845833,719.450000,5378.838889,1384.968056,276.601389,...,508.033333,1165.368056,95.806944,150.700000,0.983640,2.281202e+06,100.126389,100.134722,157.691667,157.708333
std,6.926999,1.445806,20.417913,10.225822,15.660510,15.644249,122.748055,716.705082,207.973915,47.398623,...,89.979608,215.719482,20.778324,51.624368,0.002694,8.462994e+05,5.198628,5.153814,20.414534,20.417913
min,2000.000000,1.000000,58.000000,24.000000,19.000000,17.000000,219.000000,1752.000000,390.000000,73.000000,...,145.000000,393.000000,20.000000,33.000000,0.976000,0.000000e+00,88.000000,88.000000,58.000000,58.000000
25%,2005.750000,2.000000,162.000000,81.000000,71.000000,71.000000,672.750000,5462.750000,1351.750000,263.750000,...,469.000000,1046.750000,85.000000,132.000000,0.982000,1.767692e+06,97.000000,97.000000,162.000000,162.000000
50%,2011.500000,3.000000,162.000000,81.000000,81.000000,79.000000,730.000000,5523.000000,1416.000000,281.000000,...,516.000000,1169.500000,97.000000,145.000000,0.984000,2.321000e+06,100.000000,100.000000,162.000000,162.000000
75%,2017.250000,4.000000,162.000000,81.000000,90.000000,89.000000,790.000000,5578.250000,1481.000000,302.000000,...,560.000000,1304.500000,108.000000,160.000000,0.985250,2.917326e+06,103.000000,103.000000,162.000000,162.000000
max,2023.000000,6.000000,163.000000,84.000000,116.000000,119.000000,978.000000,5770.000000,1667.000000,376.000000,...,728.000000,1687.000000,145.000000,460.000000,0.991000,4.298655e+06,125.000000,125.000000,163.000000,163.000000


In [26]:
# 📌 1️⃣ Add "Ties" Column
teams["Ties"] = teams["G"] - (teams["W"] + teams["L"])

# 📌 2️⃣ Ensure No Negative Values in "Ties"
teams.loc[teams["Ties"] < 0, "Ties"] = 0  # If somehow negative, set to 0

# 📌 3️⃣ Check for Remaining Issues
teams["W_L_Check"] = teams["W"] + teams["L"] + teams["Ties"]
teams["Games_Check"] = teams["G"]

# Find any remaining mismatches
mismatched_records = teams[teams["W_L_Check"] != teams["Games_Check"]]
print("\n🚨 Records Where Wins + Losses + Ties ≠ Games Played (Should be 0):")
display(mismatched_records)

# 📌 4️⃣ Drop Unnecessary Check Columns
teams.drop(columns=["W_L_Check", "Games_Check"], inplace=True, errors="ignore")


🚨 Records Where Wins + Losses + Ties ≠ Games Played (Should be 0):


,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro,W_L_Check,Games_Check,Ties


In [27]:
# 📌 6️⃣ Save Cleaned Data
teams.to_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/cleaned_and_processed_data/Teams_Cleaned_Final.csv", index=False)
print("✅ `Teams.csv` cleaned and saved!")

✅ `Teams.csv` cleaned and saved!


# **Salaries**

In [28]:
salaries = pd.read_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/lahman_1871-2023_csv/Salaries.csv", encoding="latin1")

# Filter to 2000+
salaries = salaries[salaries["yearID"] >= 2000]

# 📌 Display first few rows
print("📊 First 5 Rows of Teams.csv:")
display(salaries.head())

📊 First 5 Rows of Teams.csv:


,yearID,teamID,lgID,playerID,salary
12263,2000,ANA,AL,anderga01,3250000
12264,2000,ANA,AL,belchti01,4600000
12265,2000,ANA,AL,botteke01,4000000
12266,2000,ANA,AL,clemeed02,215000
12267,2000,ANA,AL,colanmi01,200000


In [29]:
#📌 Display column names
print("\n🛠 Column Names in Salaries.csv:")
display(salaries.columns)

# 📌 Compare columns to the README file (Expected columns)
expected_columns = ["yearID", "teamID", "lgID", "playerID", "salary"]

# Identify unexpected columns
unexpected_columns = [col for col in salaries.columns if col not in expected_columns]
if unexpected_columns:
    print("\n🚨 Unexpected Columns Found (Not in README):")
    display(unexpected_columns)
else:
    print("\n✅ All columns match the README file!")


🛠 Column Names in Salaries.csv:


Index(['yearID', 'teamID', 'lgID', 'playerID', 'salary'], dtype='object')


✅ All columns match the README file!


In [30]:
# 📌 Check missing values
missing_salaries = salaries.isnull().sum()
print("\n⚠️ Missing Values Per Column:")
display(missing_salaries[missing_salaries > 0])

# 📌 Check for duplicate salary records per player per year
duplicate_salaries = salaries[salaries.duplicated(subset=["yearID", "playerID"], keep=False)]
print("\n🔍 Duplicate Salary Records for the Same Player & Year (Should be 0):")
display(duplicate_salaries)

# 📌 Check for unrealistic salaries (negative values or extremely high values)
print("\n🚨 Unusual Salary Values:")
display(salaries[salaries["salary"] < 0])  # Should not have negative salaries
display(salaries[salaries["salary"] > 50_000_000])  # Checking extreme salaries


⚠️ Missing Values Per Column:


Series([], dtype: int64)


🔍 Duplicate Salary Records for the Same Player & Year (Should be 0):


,yearID,teamID,lgID,playerID,salary
12436,2000,CHA,AL,wellsbo01,210000
12743,2000,MIN,AL,wellsbo01,700000
21173,2010,OAK,AL,taverwi01,3600000
21448,2010,WAS,NL,taverwi01,400000
23374,2013,COL,NL,hernara02,3200000
23500,2013,LAN,NL,hernara02,6000000
25817,2016,COL,NL,matzety01,509500
26047,2016,MIN,AL,matzety01,545000



🚨 Unusual Salary Values:


,yearID,teamID,lgID,playerID,salary


,yearID,teamID,lgID,playerID,salary


In [31]:
# 📌 Summary statistics for salary data
print("\n📈 Salary Summary Statistics:")
display(salaries["salary"].describe())


📈 Salary Summary Statistics:


count    1.416500e+04
mean     3.114437e+06
std      4.294667e+06
min      1.655740e+05
25%      4.350000e+05
50%      1.000000e+06
75%      4.000000e+06
max      3.300000e+07
Name: salary, dtype: float64

In [32]:
# 📌 Identify Players Who Played for Multiple Teams in the Same Year
multi_team_players = salaries.groupby(["yearID", "playerID"]).size()
multi_team_players = multi_team_players[multi_team_players > 1].reset_index()
multi_team_players["MultiTeamYear"] = 1  # Flag these players

# 📌 Merge Flag Back to Original Salaries Data
salaries_cleaned = salaries.merge(multi_team_players[["yearID", "playerID", "MultiTeamYear"]],
                                  on=["yearID", "playerID"], how="left")

# 📌 Fill NaN Values in "MultiTeamYear" with 0 (for players who stayed on one team)
salaries_cleaned["MultiTeamYear"] = salaries_cleaned["MultiTeamYear"].fillna(0).astype(int)

In [33]:
#📌 Save Cleaned Data
salaries.to_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/cleaned_and_processed_data/Salaries_Cleaned_Final.csv", index=False)
print("✅ `Salaries.csv` cleaned and saved!")

✅ `Salaries.csv` cleaned and saved!


# **Awards**

In [55]:
# 📌 Load Awards Data
awards_df = pd.read_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/lahman_1871-2023_csv/AwardsPlayers.csv")

# 📌 Filter Data for Years 2000+ Only
awards_df = awards_df[awards_df["yearID"] >= 2000]

# 📌 Display First Few Rows
print("\n📊 First 5 Rows of Awards.csv:")
print(awards_df.head())


📊 First 5 Rows of Awards.csv:
    playerID                    awardID  yearID lgID  tie notes
0  poseybu01  Lou Gehrig Memorial Award    2019   ML  NaN   NaN
1  klubeco01  Lou Gehrig Memorial Award    2018   ML  NaN   NaN
2  vottojo01  Lou Gehrig Memorial Award    2017   ML  NaN   NaN
3  altuvjo01  Lou Gehrig Memorial Award    2016   ML  NaN   NaN
4  grandcu01  Lou Gehrig Memorial Award    2015   ML  NaN   NaN


In [56]:
# 📌 Display Column Names
print("\n🛠 Column Names:")
print(awards_df.columns)


🛠 Column Names:
Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')


In [57]:
# 📌 Get Unique Awards to Choose From
print("\n🏆 Unique Award Names in Dataset:")
print(awards_df['awardID'].unique())


🏆 Unique Award Names in Dataset:
['Lou Gehrig Memorial Award' 'Most Valuable Player' 'NLCS MVP'
 'Outstanding DH Award' 'Pitching Triple Crown' 'Platinum Glove'
 'TSN All-Star' 'TSN Major League Player of the Year'
 'TSN Pitcher of the Year' 'TSN Reliever of the Year' 'World Series MVP'
 'Reliever of the Year Award' 'Silver Slugger' 'ALCS MVP'
 'All-Star Game MVP' 'Roberto Clemente Award' 'Rookie of the Year'
 'Gold Glove' 'Babe Ruth Award' 'Comeback Player of the Year'
 'Cy Young Award' 'Hank Aaron Award' 'Hutch Award' 'SIlver Slugger'
 'Branch Rickey Award' 'TSN Fireman of the Year' 'Triple Crown']


In [58]:
# 📌 Filter Only for Selected Awards
selected_awards = [
    "Most Valuable Player", "Cy Young Award", "Rookie of the Year",
    "Gold Glove", "Silver Slugger", "Pitching Triple Crown", 'Platinum Glove', 'Triple Crown'
]
awards_df = awards_df[awards_df["awardID"].isin(selected_awards)]

In [59]:
# 📌 Check for Missing Values
print("\n⚠️ Missing Values Per Column:")
print(awards_df.isnull().sum())


⚠️ Missing Values Per Column:
playerID       0
awardID        0
yearID         0
lgID           0
tie         1043
notes        176
dtype: int64


In [61]:
# 📌 Handle Missing Values
awards_df["notes"].fillna("N/A", inplace=True)  # Keep notes for position-specific awards
awards_df["tie"].fillna("No", inplace=True)  # Mark all missing tie values as "No"

In [62]:
# 📌 Check the Cleaned Data
print("\n✅ Cleaned Awards Data Preview:")
print(awards_df.head())


✅ Cleaned Awards Data Preview:
     playerID                awardID  yearID lgID tie notes
8   acunaro01   Most Valuable Player    2023   NL  No   N/A
9   ohtansh01   Most Valuable Player    2023   AL  No   N/A
10  judgeaa01   Most Valuable Player    2022   AL  No   N/A
11  goldspa01   Most Valuable Player    2022   NL  No   N/A
16  biebesh01  Pitching Triple Crown    2020   AL  No   N/A


In [64]:
# 📌 Save Cleaned Data
awards_df.to_csv("/Users/colbyreichenbach/Desktop/Portfolio/MLB_SQL/lahman_data/cleaned_and_processed_data/Awards_Cleaned_Final.csv", index=False)
print("\n✅ Cleaned Awards Data Saved!")


✅ Cleaned Awards Data Saved!
